0. Read in all json files. Done.

First pass:
1. Read asm file. Done.
2. Remove whitespace. Done.
3. Remove comments. Done.
4. Find label symbols and add line number to table. Done.

Second pass:
1. Translate each line to binary instructions. Done.
2. After every 10 lines, write to file and clear string. Done.

Helper functions:
1. Read c-instruction. Done.
2. variable symbols. Done.
3. read a-instruction. Done.

In [1]:
import json
import re
import os

In [10]:
with open('symbol_table.json') as f:
    symbol_table = json.load(f)
with open('c_instructions.json') as f:
    c_instructions = json.load(f)
with open('d_table.json') as f:
    d_table = json.load(f)
with open('j_table.json') as f:
    j_table = json.load(f)

In [3]:
def first_pass(f):
    line_number = 0
    for line in f:
        line = re.sub('\s+', '', line) # remove whitespace
        line = re.sub('\/\/.*', '', line) # remove comments
        if len(line) > 0:
            if line[0] == '(':
                if line[1:-1] not in symbol_table:
                    symbol_table[line[1:-1]] = line_number
            else:
                line_number += 1

In [4]:
d_table

{'0': '000',
 'M': '001',
 'D': '010',
 'DM': '011',
 'A': '100',
 'AM': '101',
 'AD': '110',
 'ADM': '111'}

In [36]:
asm_file = 'Add.asm'

In [38]:
with open(asm_file) as f:
    first_pass(f)

In [40]:
curr_addr = 16

In [51]:
def read_a_instruction(line):
    addr = line[1:]
    try:
        a = int(addr)
    except ValueError:
        if addr not in symbol_table:
            global curr_addr
            symbol_table[addr] = curr_addr
            curr_addr += 1
        a = symbol_table[addr]
    return f'{int(bin(a)[2:]):016d}\n'

In [42]:
def read_c_instruction(line):
    c_instr = line.split(';')
    if len(c_instr) > 1:
        j = c_instr[1]
    else:
        j = '0'
    
    c_instr = c_instr[0].split('=')
    if len(c_instr) > 1:
        d = ''.join(sorted(c_instr[0]))
        c = c_instr[1]
    else:
        d = '0'
        c = c_instr[0]
    
    return '111' + c_instructions[c]['a'] + c_instructions[c]['c'] + d_table[d] + j_table[j] + '\n'

In [52]:
def second_pass(f):
    hack_file = asm_file[:-4] + '.hack'
    if os.path.exists(hack_file):
        os.remove(hack_file)
    line_number = 0
    instr = ''
    for line in f:
        line = re.sub('\s+', '', line) # remove whitespace
        line = re.sub('\/\/.*', '', line) # remove comments
        if len(line) > 0:
            if line[0] == '(':
                continue
            elif line[0] == '@':
                instr += read_a_instruction(line)
                line_number += 1
            else:
                instr += read_c_instruction(line)
                line_number += 1
            if line_number == 10:
                line_number = 0
                with open(hack_file, 'a+') as h:
                    h.write(instr)
                instr = ''
    with open(hack_file, 'a+') as h:
        h.write(instr)

In [58]:
with open(asm_file) as f:
    second_pass(f)

'+DM'